In [1]:
import pandas as pd

#### Check value counts for each `gender` column 

In [2]:
movies_2011_and_beyond = pd.read_csv('my_data/movies_2011_and_beyond.csv')
movies_2011_and_beyond.head()

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Fast Five,Neal H. Moritz,male,NaN,NaN,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
1,Albert Nobbs,Rodrigo García,male,NaN,NaN,Glenn Close,female,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
2,Albert Nobbs,Rodrigo García,male,NaN,NaN,George Moore,male,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
3,300: Rise of an Empire,Noam Murro,male,NaN,NaN,Kurt Johnstad,male,NaN,NaN,Thomas Tull,male,110000000.0,337580051.0,tt1253863
4,Fast Five,Neal H. Moritz,male,NaN,NaN,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343


### Custom Functions
* add docstrings / make pretty for all finalized functions that are used

In [3]:
# check for all unique values in gender columns, will use my column names within function since they're all the same

def check_unique_gender_values(df):
    gender_columns = ['directorGender', 'writerGender', 'screenwriterGender', 'executiveproducerGender', 'producerGender']
    for col in gender_columns:
        print(col)
        print(df[col].unique())

In [4]:
check_unique_gender_values(movies_2011_and_beyond)

directorGender
['male' 'female' nan 'transgender female' 'non-binary' 'transgender male']
writerGender
[nan 'male']
screenwriterGender
['male' 'female' nan 'transgender female' 'non-binary']
executiveproducerGender
['male' nan 'female']
producerGender
['male' 'female' 'transgender female' nan]


In [6]:
# investigate writerGender because this should probably be dropped and changed to screenwriter
movies_2011_and_beyond.writerName.unique()

array([nan, 'Slavoj Žižek', 'Rana Abrar'], dtype=object)

In [7]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.writerName == 'Rana Abrar']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
15766,Black Briefcase: The Nuclear Trigger,Rana Abrar,male,Rana Abrar,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt7903992
16053,Son Of Kashmir Burhan,Rana Abrar,male,Rana Abrar,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt7675680
16390,The Evil Marriage,Rana Abrar,male,Rana Abrar,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt8614336


In [ ]:
# check to see if these movies are in Bechdel dataset - if not, all 3 will be dropped because there is not much useful
# information for these movies

In [8]:
bechdel_df = pd.read_csv('my_data/bechdel_test_movies.csv')
bechdel_df.head()

,year,title,score,passing,imdb_id,imdb_link
0,2019,Alita: Battle Angel,3,1,tt0437086,http://us.imdb.com/title/tt0437086/
1,2019,American Woman,3,1,tt9109492,http://us.imdb.com/title/tt9109492/
2,2019,Close,3,1,tt5316540,http://us.imdb.com/title/tt5316540/
3,2019,Cold Pursuit,1,0,tt5719748,http://us.imdb.com/title/tt5719748/
4,2019,How to Train Your Dragon: The Hidden World,2,0,tt2386490,http://us.imdb.com/title/tt2386490/


In [16]:
# could turn this imdb_id finder into a function
rana_abrar_imdb_ids = []
for movie in movies_2011_and_beyond.loc[movies_2011_and_beyond.writerName == 'Rana Abrar'].values:
    rana_abrar_imdb_ids.append(movie[-1])
rana_abrar_imdb_ids

['tt7903992', 'tt7675680', 'tt8614336']

In [14]:
def locate_rows_by_imdb_ids(dataset, name_of_imdb_id_col, imdb_id_list):
    selected_rows = []
    for imdb_id in imdb_id_list:
        row = dataset.loc[dataset[name_of_imdb_id_col] == imdb_id]
        selected_rows.append(row)
    return selected_rows

In [19]:
locate_rows_by_imdb_ids(bechdel_df, name_of_imdb_id_col='imdb_id', imdb_id_list=rana_abrar_imdb_ids)

[Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: []]

In [21]:
# confirm this function is actually saying there are no matching movies in Bechdel df rather than just being a 
# broken function

bechdel_df.loc[bechdel_df.imdb_id == 'tt7903992']

,year,title,score,passing,imdb_id,imdb_link


In [22]:
bechdel_df.loc[bechdel_df.title == 'The Evil Marriage']

,year,title,score,passing,imdb_id,imdb_link


In [23]:
# check to see if function works on movies that are definitely in both lists

movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Alita: Battle Angel']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
8292,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,James Cameron,male,NaN,NaN,James Cameron,male,170000000.0,359700000.0,tt0437086
8299,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,Laeta Kalogridis,female,NaN,NaN,James Cameron,male,170000000.0,359700000.0,tt0437086
8307,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,James Cameron,male,NaN,NaN,Robert Rodriguez,male,170000000.0,359700000.0,tt0437086
8314,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,Laeta Kalogridis,female,NaN,NaN,Robert Rodriguez,male,170000000.0,359700000.0,tt0437086
8331,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,James Cameron,male,NaN,NaN,Jon Landau,male,170000000.0,359700000.0,tt0437086
8333,Alita: Battle Angel,Robert Rodriguez,male,NaN,NaN,Laeta Kalogridis,female,NaN,NaN,Jon Landau,male,170000000.0,359700000.0,tt0437086


In [24]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Close']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
15209,Close,Vicky Jewson,female,NaN,NaN,Vicky Jewson,female,NaN,NaN,NaN,NaN,NaN,NaN,tt5316540


In [25]:
def locate_imdb_ids(df, column_to_search, value_list_to_search, imdb_id_column):
    rows_to_search = []
    imdb_ids = []
    for val in value_list_to_search:
        row = df.loc[df[column_to_search] == val]
        rows_to_search.append(row)
    for row in rows_to_search.values:
        print(row)

In [27]:
# okay that didn't really work

In [30]:
def locate_rows_in_df_by_values(df, column_to_search, value_list_to_search):
    for val in value_list_to_search:
        print(df.loc[df[column_to_search] == val])

In [32]:
movies_to_search = ['Close', 'Alita: Battle Angel', 'American Woman']
locate_rows_in_df_by_values(movies_2011_and_beyond, 'movieTitle', movies_to_search)

      movieTitle  directorName directorGender writerName writerGender  \
15209      Close  Vicky Jewson         female        NaN          NaN   

      screenwriterName screenwriterGender executiveproducerName  \
15209     Vicky Jewson             female                   NaN   

      executiveproducerGender producerName producerGender  budgetAmount  \
15209                     NaN          NaN            NaN           NaN   

       boxofficeAmount     imdbId  
15209              NaN  tt5316540  
               movieTitle      directorName directorGender writerName  \
8292  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8299  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8307  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8314  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8331  Alita: Battle Angel  Robert Rodriguez           male        NaN   
8333  Alita: Battle Angel  Robert Rodriguez           mal

In [34]:
test_imdb_ids = ['tt0437086', 'tt5316540']
locate_rows_by_imdb_ids(movies_2011_and_beyond, 'imdbId', test_imdb_ids)

[               movieTitle      directorName directorGender writerName  \
 8292  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8299  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8307  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8314  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8331  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 8333  Alita: Battle Angel  Robert Rodriguez           male        NaN   
 
      writerGender  screenwriterName screenwriterGender executiveproducerName  \
 8292          NaN     James Cameron               male                   NaN   
 8299          NaN  Laeta Kalogridis             female                   NaN   
 8307          NaN     James Cameron               male                   NaN   
 8314          NaN  Laeta Kalogridis             female                   NaN   
 8331          NaN     James Cameron               male                   N

In [35]:
locate_rows_by_imdb_ids(bechdel_df, 'imdb_id', test_imdb_ids)

[   year                title  score  passing    imdb_id  \
 0  2019  Alita: Battle Angel      3        1  tt0437086   
 
                              imdb_link  
 0  http://us.imdb.com/title/tt0437086/  ,
    year  title  score  passing    imdb_id                            imdb_link
 2  2019  Close      3        1  tt5316540  http://us.imdb.com/title/tt5316540/]

In [33]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'American Woman']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId


In [36]:
# well the outputs don't look pretty, but at least they work!

In [37]:
# the whole point of all that was just to see if I could drop the writerName and writerGender columns

In [38]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.writerName == 'Slavoj Žižek']

,movieTitle,directorName,directorGender,writerName,writerGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
11007,The Pervert's Guide to Ideology,Sophie Fiennes,female,Slavoj Žižek,male,Slavoj Žižek,male,NaN,NaN,NaN,NaN,NaN,NaN,tt2152198


In [39]:
# lol

In [40]:
# check to see if this movie is in the bechdel dataset
locate_rows_by_imdb_ids(bechdel_df, 'imdb_id', 'tt2152198')

[Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: [], Empty DataFrame
 Columns: [year, title, score, passing, imdb_id, imdb_link]
 Index: []]

In [41]:
bechdel_df.loc[bechdel_df.title == 'The Pervert\'s Guide to Ideology']

,year,title,score,passing,imdb_id,imdb_link


In [42]:
# drop writerName and writerGender from movies_200 df
movies_2011_and_beyond.drop(columns=['writerName', 'writerGender'], axis=1, inplace=True)

In [43]:
movies_2011_and_beyond.head()

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
1,Albert Nobbs,Rodrigo García,male,Glenn Close,female,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
2,Albert Nobbs,Rodrigo García,male,George Moore,male,NaN,NaN,Bonnie Curtis,female,NaN,NaN,tt1602098
3,300: Rise of an Empire,Noam Murro,male,Kurt Johnstad,male,NaN,NaN,Thomas Tull,male,110000000.0,337580051.0,tt1253863
4,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343


#### Convert gender columns to a boolean `FTNB` column to include cis and transgender females, transgender males, and non-binary people
* Need to be able to drop duplicate rows so that there is only one row for each movie
* Each movie row would indicate whether one of its directors, writers, or producers is of an underrepresented gender
* Would like an additional column for presence of underrepresented gender in ANY of those listed positions

In [44]:
# test subset of dataframe - will use index slicing for sample df 
movies_2011_and_beyond.loc[movies_2011_and_beyond.directorGender == 'transgender male']

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200


In [47]:
test_indices = []
test_indices.append(14917)
test_indices.append(15646)

In [48]:
test_indices

[14917, 15646]

In [49]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.screenwriterGender == 'non-binary']

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890
3750,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Sebastian Dungan,male,NaN,NaN,tt2312890
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462


In [50]:
test_indices.append(3747)
test_indices.append(11491)

In [65]:
# movies_2011_and_beyond.loc[movies_2011_and_beyond.producerGender == 'transgender female']

In [52]:
# the list above contains movies with transgender females, females, and males, so can use these for test df
test_indices.append(8957)
test_indices.append(9060)
test_indices.append(8961)
test_indices.append(7410)

In [53]:
test_indices

[14917, 15646, 3747, 11491, 8957, 9060, 8961, 7410]

In [56]:
movies_2011_and_beyond[1000:1200]

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
1000,Thor,Kenneth Branagh,male,Zack Stentz,male,Stan Lee,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1001,Thor,Kenneth Branagh,male,Larry Lieber,male,Patricia Whitcher,female,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1002,Thor,Kenneth Branagh,male,Zack Stentz,male,Patricia Whitcher,female,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1003,Thor,Kenneth Branagh,male,Don Payne,male,Stan Lee,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1004,Thor,Kenneth Branagh,male,Mark Protosevich,male,Louis D'Esposito,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1005,Thor,Kenneth Branagh,male,Mark Protosevich,male,Alan Fine,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1006,Thor,Kenneth Branagh,male,Don Payne,male,Patricia Whitcher,female,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1007,Thor,Kenneth Branagh,male,Mark Protosevich,male,David Maisel,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1008,Thor,Kenneth Branagh,male,Jack Kirby,male,Louis D'Esposito,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369
1009,Thor,Kenneth Branagh,male,Jack Kirby,male,Alan Fine,male,Kevin Feige,male,150000000.0,8.531648e+06,tt0800369


In [59]:
# check all instances of Fast Five, Get the Gringo, and Girl Walks into a Bar 

In [60]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Fast Five']

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
4,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343
5,Fast Five,Justin Lin,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
9,Fast Five,Justin Lin,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
19,Fast Five,Vin Diesel,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
21,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343
22,Fast Five,Vin Diesel,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343
27,Fast Five,Gary Scott Thompson,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
34,Fast Five,Gary Scott Thompson,male,Chris Morgan,male,Justin Lin,male,Neal H. Moritz,male,NaN,626100000.0,tt1596343


In [61]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Get the Gringo']

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
11,Get the Gringo,Adrian Grunberg,male,Mel Gibson,male,NaN,NaN,Mel Gibson,male,NaN,NaN,tt1567609
25,Get the Gringo,Adrian Grunberg,male,Mel Gibson,male,NaN,NaN,Bruce Davey,male,NaN,NaN,tt1567609


In [62]:
# there is a third producer for Get the Gringo - Stacy Perskie, cisgender male - so this movie would still be 
# good to use for test

In [63]:
movies_2011_and_beyond.loc[movies_2011_and_beyond.movieTitle == 'Girl Walks into a Bar']

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
1181,Girl Walks into a Bar,Sebastian Gutierrez,male,Sebastian Gutierrez,male,NaN,NaN,Steve Bing,male,NaN,NaN,tt1682246


In [ ]:
# Girl Walks into a Bar is edited by female Lisa Bromwell, but all the other producers are male
# will have to add other roles like film editor, cinematographer, etc as later wish list for project

In [64]:
# add some movies with all cisgender males to test df
test_indices.append(0) # Fast Five
test_indices.append(18) # Fast Five
test_indices.append(11) # Get the Gringo
test_indices.append(25) # Get the Gringo
test_indices.append(1181) # Girl Walks into a Bar

In [66]:
gender_columns = ['directorGender', 'screenwriterGender', 'executiveproducerGender', 'producerGender']
gender_values = ['female', 'male', 'non-binary', 'transgender female', 'transgender male']

In [67]:
test_indices

[14917, 15646, 3747, 11491, 8957, 9060, 8961, 7410, 0, 18, 11, 25, 1181]

In [68]:
test_df = movies_2011_and_beyond.iloc[test_indices]

In [69]:
test_df

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462
8957,Cloud Atlas,Lilly Wachowski,transgender female,Lilly Wachowski,female,Pearry Teo,male,Lana Wachowski,transgender female,NaN,NaN,tt1371111
9060,Cloud Atlas,Lilly Wachowski,female,Lilly Wachowski,transgender female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111
8961,Cloud Atlas,Tom Tykwer,male,Lilly Wachowski,female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111
7410,Turning,Anohni,transgender female,Anohni,transgender female,NaN,NaN,Anohni,transgender female,NaN,NaN,tt2219894
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343


In [70]:
test_df.directorGender.str.contains('female')

14917    False
15646    False
3747     False
11491     True
8957      True
9060      True
8961     False
7410      True
0        False
18       False
11       False
25       False
1181     False
Name: directorGender, dtype: bool

In [75]:
for idx, value in enumerate(test_df.directorGender.str.contains('female')):
    print(idx, value)

0 False
1 False
2 False
3 True
4 True
5 True
6 False
7 True
8 False
9 False
10 False
11 False
12 False


In [76]:
# could possibly use enumerate, depending on how this deals with nan values
test_df.executiveproducerGender.str.contains('male')

14917     NaN
15646     NaN
3747      NaN
11491     NaN
8957     True
9060     True
8961     True
7410      NaN
0        True
18       True
11        NaN
25        NaN
1181      NaN
Name: executiveproducerGender, dtype: object

In [79]:
for idx, value in enumerate(test_df.directorGender.str.contains('female|non-binary|transgender')):
    if value:
        print(idx)

0
1
2
3
4
5
7


In [82]:
director_fnbt = [1 if val == True else 0 for val in test_df.directorGender.str.contains('female|non-binary|transgender')]
                                                              

In [83]:
director_fnbt

[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]

In [84]:
test_df.loc['director_fnbt'] = director_fnbt

/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [85]:
test_df

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId,director_fnbt
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150,1
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200,1
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890,1
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462,1
8957,Cloud Atlas,Lilly Wachowski,transgender female,Lilly Wachowski,female,Pearry Teo,male,Lana Wachowski,transgender female,NaN,NaN,tt1371111,1
9060,Cloud Atlas,Lilly Wachowski,female,Lilly Wachowski,transgender female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,1
8961,Cloud Atlas,Tom Tykwer,male,Lilly Wachowski,female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,0
7410,Turning,Anohni,transgender female,Anohni,transgender female,NaN,NaN,Anohni,transgender female,NaN,NaN,tt2219894,1
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0


In [89]:
test_df.drop('new_col', axis=1, inplace=True)

/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [90]:
new_writer_values = [1 if val == True else 0 for val in test_df.screenwriterGender.str.contains('female|non-binary|transgender')]

In [92]:
test_df['screenwriter_fnbt'] = new_writer_values

/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [93]:
test_df

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId,director_fnbt
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150,1
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200,1
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890,1
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462,1
8957,Cloud Atlas,Lilly Wachowski,transgender female,Lilly Wachowski,female,Pearry Teo,male,Lana Wachowski,transgender female,NaN,NaN,tt1371111,1
9060,Cloud Atlas,Lilly Wachowski,female,Lilly Wachowski,transgender female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,1
8961,Cloud Atlas,Tom Tykwer,male,Lilly Wachowski,female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,0
7410,Turning,Anohni,transgender female,Anohni,transgender female,NaN,NaN,Anohni,transgender female,NaN,NaN,tt2219894,1
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0


In [94]:
# omg what is going on

In [95]:
test_df.drop('screenwriter_fnbt', axis=0, inplace=True)

/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [96]:
test_df

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId,director_fnbt
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150,1
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200,1
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890,1
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462,1
8957,Cloud Atlas,Lilly Wachowski,transgender female,Lilly Wachowski,female,Pearry Teo,male,Lana Wachowski,transgender female,NaN,NaN,tt1371111,1
9060,Cloud Atlas,Lilly Wachowski,female,Lilly Wachowski,transgender female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,1
8961,Cloud Atlas,Tom Tykwer,male,Lilly Wachowski,female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,0
7410,Turning,Anohni,transgender female,Anohni,transgender female,NaN,NaN,Anohni,transgender female,NaN,NaN,tt2219894,1
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0


In [97]:
new_writer_values

[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [98]:
test_df['writer_fnbt'] = new_writer_values

/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [99]:
test_df

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150,1,0
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200,1,0
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890,1,1
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462,1,1
8957,Cloud Atlas,Lilly Wachowski,transgender female,Lilly Wachowski,female,Pearry Teo,male,Lana Wachowski,transgender female,NaN,NaN,tt1371111,1,1
9060,Cloud Atlas,Lilly Wachowski,female,Lilly Wachowski,transgender female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,1,1
8961,Cloud Atlas,Tom Tykwer,male,Lilly Wachowski,female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,0,1
7410,Turning,Anohni,transgender female,Anohni,transgender female,NaN,NaN,Anohni,transgender female,NaN,NaN,tt2219894,1,1
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0,0
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0,0


In [103]:
new_exec_values = [1 if val == True else 0 for val in test_df.executiveproducerGender.str.contains('female|non-binary|transgender')]
new_producer_values = [1 if val == True else 0 for val in test_df.producerGender.str.contains('female|non-binary|transgender')]

In [104]:
test_df['exec_fnbt'] = new_exec_values
test_df['producer_fnbt'] = new_producer_values

/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [105]:
test_df

,movieTitle,directorName,directorGender,screenwriterName,screenwriterGender,executiveproducerName,executiveproducerGender,producerName,producerGender,budgetAmount,boxofficeAmount,imdbId,director_fnbt,writer_fnbt,exec_fnbt,producer_fnbt
14917,Strong Island,Yance Ford,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt5873150,1,0,0,0
15646,A Kid Like Jake,Silas Howard,transgender male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt6884200,1,0,0,0
3747,Afternoon Delight,Jill Soloway,non-binary,Jill Soloway,non-binary,NaN,NaN,Jen Chaiken,female,NaN,NaN,tt2312890,1,1,0,1
11491,My Prairie Home,Chelsea McMullan,female,Rae Spoon,non-binary,NaN,NaN,NaN,NaN,NaN,NaN,tt3203462,1,1,0,0
8957,Cloud Atlas,Lilly Wachowski,transgender female,Lilly Wachowski,female,Pearry Teo,male,Lana Wachowski,transgender female,NaN,NaN,tt1371111,1,1,0,1
9060,Cloud Atlas,Lilly Wachowski,female,Lilly Wachowski,transgender female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,1,1,0,1
8961,Cloud Atlas,Tom Tykwer,male,Lilly Wachowski,female,Pearry Teo,male,Lilly Wachowski,transgender female,NaN,NaN,tt1371111,0,1,0,1
7410,Turning,Anohni,transgender female,Anohni,transgender female,NaN,NaN,Anohni,transgender female,NaN,NaN,tt2219894,1,1,0,1
0,Fast Five,Neal H. Moritz,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0,0,0,0
18,Fast Five,Chris Morgan,male,Chris Morgan,male,Justin Lin,male,Vin Diesel,male,NaN,626100000.0,tt1596343,0,0,0,0


In [ ]:
# could just add values for overall values, then replace any value that doesn't equal 0 to 1
overall_values = [1 if val == True else 0 for val in test_df.screenwriterGender.str.contains('female|non-binary|transgender')]